## Part1- Target-Disease pairs 

###  1) Using FTP download json files evidence, disease and target
###  2) Extract needed columns: diseaseId, targetId, score and store in seprate table
###  3) Run the main function which returns a table with target-disease pairs with median and 3maxs
###  4) Join the resultant dataset with target and disease dataset on disease.id and target.id fields and add `target.approvedSymbol` and `disease.name` to the table
###  5) Covert the table to json format and return 

###   

In [1]:
from ftplib import FTP
import pandas as pd

In [2]:
import statistics
import json
import math

### Step 1

In [4]:
def getFile(ftp, filename):
    try:
        ftp.retrbinary("RETR " + filename ,open(filename, 'wb').write)
    except:
        print( "Error")


In [ ]:
ftp = FTP('ftp.ebi.ac.uk')   # connect to host, default port
ftp.login()

In [ ]:
# Json file from evidence=evs part0000
# ftp.cwd("pub/databases/opentargets/platform/21.11/output/etl/json/evidence/sourceId=eva/")
# getFile(ftp,"part-00000-4134a310-5042-4942-82ed-565f3d91eddd.c000.json")


In [ ]:
# Disease file
# ftp.cwd("pub/databases/opentargets/platform/21.11/output/etl/json/diseases/")
# getFile(ftp,"part-00000-773deead-54e9-4934-b648-b26a4bbed763-c000.json")

In [ ]:
#Target file
# ftp.cwd("pub/databases/opentargets/platform/21.11/output/etl/json/targets/")
# getFile(ftp,"part-00000-9befc20b-ce53-4029-bd62-39c5b631aa3f-c000.json")

In [6]:
#This is assuming that all files are in same directory as the current jupyter file 
df_evidence = pd.read_json('part-00000-4134a310-5042-4942-82ed-565f3d91eddd.c000.json',lines=True)
df_diseases = pd.read_json('part-00000-773deead-54e9-4934-b648-b26a4bbed763-c000.json',lines=True)
df_targets  = pd.read_json('part-00000-9befc20b-ce53-4029-bd62-39c5b631aa3f-c000.json',lines=True)

### Part 2

In [7]:
df_evidence=df_evidence[['targetId','diseaseId','score']]

In [8]:
df_evidence

,targetId,diseaseId,score
0,ENSG00000168646,Orphanet_300576,0.02
1,ENSG00000183873,Orphanet_130,0.32
2,ENSG00000101605,EFO_0000538,0.32
3,ENSG00000169247,EFO_0004143,0.02
4,ENSG00000096696,Orphanet_247,0.72
...,...,...,...
5574,ENSG00000095002,Orphanet_140162,0.02
5575,ENSG00000134571,HP_0001626,0.95
5576,ENSG00000166147,EFO_0009299,0.32
5577,ENSG00000141510,Orphanet_524,0.32


### Step 3

In [9]:
lis=[]

def transform(dataset):
    diseases = dataset['diseaseId'].unique()
    
    for d in diseases:
        temp_df  = dataset[dataset['diseaseId']==d]
        targets  = temp_df['targetId'].unique()
        
        for t in targets:
            scores = sorted(temp_df[temp_df['targetId']==t]["score"],reverse=True)
            median = round(statistics.median(scores),2)
            if (len(scores)<3) :
                if len(scores)==2: max3_ = [round(scores[0],2),round(scores[1],2),0]
                else: max3_ = [round(scores[0],2),0,0]
            else: max3_ = [round(scores[0],2),round(scores[1],2),round(scores[2],2)]
            dic_ = {'disease':d,'target':t, 'median':median, 'top3':max3_}
            lis.append(dic_)
        
    
    return lis

transform(df_evidence)

[{'disease': 'Orphanet_300576',
  'target': 'ENSG00000168646',
  'median': 0.32,
  'top3': [0.32, 0.32, 0.32]},
 {'disease': 'Orphanet_130',
  'target': 'ENSG00000183873',
  'median': 0.02,
  'top3': [0.92, 0.32, 0.32]},
 {'disease': 'Orphanet_130',
  'target': 'ENSG00000185313',
  'median': 0.02,
  'top3': [0.02, 0.02, 0]},
 {'disease': 'Orphanet_130',
  'target': 'ENSG00000153956',
  'median': 0.32,
  'top3': [0.32, 0, 0]},
 {'disease': 'Orphanet_130',
  'target': 'ENSG00000138622',
  'median': 0.17,
  'top3': [0.32, 0.32, 0.02]},
 {'disease': 'Orphanet_130',
  'target': 'ENSG00000176076',
  'median': 0.02,
  'top3': [0.02, 0, 0]},
 {'disease': 'Orphanet_130',
  'target': 'ENSG00000152642',
  'median': 0.61,
  'top3': [0.9, 0.32, 0]},
 {'disease': 'Orphanet_130',
  'target': 'ENSG00000165995',
  'median': 0.3,
  'top3': [0.3, 0, 0]},
 {'disease': 'Orphanet_130',
  'target': 'ENSG00000163681',
  'median': 0.02,
  'top3': [0.02, 0, 0]},
 {'disease': 'Orphanet_130',
  'target': 'ENSG000

In [10]:
# main_df=pd.json_normalize(lis).sort_values('median',ascending=False)
main_df=pd.json_normalize(lis).sort_values('median',ascending=False)

#To show all rows
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(main_df)

In [11]:
main_df

,disease,target,median,top3
1445,Orphanet_97120,ENSG00000154864,0.95,"[0.95, 0, 0]"
52,Orphanet_79189,ENSG00000124587,0.95,"[0.95, 0, 0]"
2117,EFO_0007409,ENSG00000036473,0.95,"[0.95, 0, 0]"
1374,Orphanet_132,ENSG00000114200,0.95,"[0.95, 0, 0]"
1562,Orphanet_845,ENSG00000213614,0.95,"[0.95, 0, 0]"
...,...,...,...,...
2200,Orphanet_418,ENSG00000231852,0.00,"[0.0, 0, 0]"
965,Orphanet_1572,ENSG00000240505,0.00,"[0.0, 0, 0]"
1600,HP_0002269,ENSG00000196924,0.00,"[0.0, 0, 0]"
1680,Orphanet_912,ENSG00000124587,0.00,"[0.0, 0, 0]"


### Step 4

### Step 5

In [ ]:
main_df.to_json('./export1.json',orient='records')


## Part2- Target-Target pairs 


In [12]:
main_df.sort_values('disease')

,disease,target,median,top3
2227,EFO_0000095,ENSG00000141510,0.70,"[0.7, 0, 0]"
885,EFO_0000181,ENSG00000141510,0.70,"[0.7, 0, 0]"
2013,EFO_0000222,ENSG00000245848,0.32,"[0.32, 0, 0]"
2072,EFO_0000239,ENSG00000204370,0.02,"[0.02, 0, 0]"
2071,EFO_0000239,ENSG00000117118,0.32,"[0.32, 0, 0]"
...,...,...,...,...
1177,Orphanet_99948,ENSG00000104381,0.02,"[0.02, 0, 0]"
711,Orphanet_99949,ENSG00000169247,0.32,"[0.32, 0.32, 0.02]"
786,Orphanet_99951,ENSG00000158887,0.32,"[0.32, 0, 0]"
1753,Orphanet_99956,ENSG00000133812,0.02,"[0.02, 0, 0]"


In [13]:
def targetPairs(main_df):
    
    main_df=main_df.sort_values('disease')
    disease=main_df['disease'].to_list()
    
    disease_start = ""  #To keep track of disease where loop starts at
    start_index = 0     #Index of las visited disease
    count = 0           #Count to track how many consecutive diseases 
    track = []          #List to keep track of num of consecutive diseases
    
    for d in range (len(disease)):
        if d == 0:
            disease_start = disease[d]
            start_index = d
        elif disease[d] !=disease_start:
            if count != 0:
                track.append(count+1)
            disease_start = disease[d]
            start_index = d
            count = 0
        else:
            count += 1
            start_index = d
    
    total_pairs=0
    for t in track:
        if t==2:
            total_pairs+=1
        else:
            num=math.factorial(t)/(2*math.factorial(t-2))
            total_pairs+=num 
    
    return total_pairs
    
    

In [14]:
print("Num of target-target pairs which share a connection to at least two diseases are: ")
print(int(targetPairs(main_df)))

Num of target-target pairs which share a connection to at least two diseases are: 
6549
